# DAISY RNAseq data projection

This notebook project RNA sequencing data into a Latent Space representing gene modules that are co expressed given various conditions, cell types, ... 

Sakaiza Rasolofomanana Rajery

12/19/2024

## load packages

In [16]:
library(tidyverse,warn.conflicts=FALSE)
library(reticulate)
library(MASS,warn.conflicts=FALSE)
library(devtools,warn.conflicts=FALSE)
library(PLIER,warn.conflicts=FALSE)
#install_github("wgmao/PLIER")

In [17]:
# srouce plier_util to get function GetnewdataB 
source('scripts/plier_util.R')

In [18]:
# use GetOrderedRowNormEM function from Marc to order matrices
GetOrderedRowNormEM <- function(exprs.mat, plier.model) {

    require(PLIER)
    
    # Z-score normalization
    exprs.norm <- rowNorm(exprs.mat)
    exprs.norm <- na.omit(exprs.norm)
    
    z.mat <- plier.model$Z
    genes.in.model <- rownames(z.mat)
    genes_in_exprs <- rownames(exprs.norm)
    
    # Find the common genes
    common_genes <- intersect(genes_in_exprs, genes.in.model)
    
    # Filter the matrices based on the common genes
    exprs.norm.filtered <- exprs.norm[common_genes, , drop = FALSE]
    z.mat.filtered <- z.mat[common_genes, , drop = FALSE]
        
  # Update the plier.model with the new Z matrix
  plier.model$Z <- z.mat.filtered

  # Return the updated plier.model and the filtered exprs.norm
  list(plier.model = plier.model, exprs.norm.filtered = exprs.norm.filtered)

}

## load data: RNA seq, Matrices, name to ID, metada

### RNA_seq data

In [19]:
# path to rna dataset
path_to_rna <- "/Users/rasolofs/Library/CloudStorage/OneDrive-TheUniversityofColoradoDenver/DAISY RNA Phenoplier"
#raw value at visit 1 and 2
#RNAseq_1_Raw <- readRDS("RNA_Visit_1.RDS")
#RNAseq_2_Raw <- readRDS("RNA_Visit_2.RDS")

#r-log adjusted for age, sex and ancestry at visit 1 and 2
RNAseq_1_res <- as.matrix(readRDS(paste0(path_to_rna,"/RNA_Visit_1_Residuals.RDS")))
#RNAseq_2_res <- readRDS("RNA_Visit_2_Residuals.RDS")

#RNA_annot <- readRDS("Expression_Annotation.RDS")
#RNA_pheno <- read_csv("DAISY_RNASeq_Phenotype_Deidentified.csv")

str(RNAseq_1_res)
head(RNAseq_1_res,5)
dim(RNAseq_1_res)

 num [1:22236, 1:136] 0.031 -0.0478 -0.0553 0.0398 -0.1755 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:22236] "ENSG00000000003" "ENSG00000000419" "ENSG00000000457" "ENSG00000000460" ...
  ..$ : chr [1:136] "00041-0" "00110-0" "00139-0" "00159-0" ...


00041-0     00110-0     00139-0    00159-0      00174-0    
ENSG00000000003  0.03095809 -0.07856905  0.1918862 -0.171697815 -0.05682264
ENSG00000000419 -0.04781262  0.22526729  0.1496638  0.008030305  0.07860273
ENSG00000000457 -0.05528318 -0.06535310  0.1215783  0.042311336 -0.13641096
ENSG00000000460  0.03975773 -0.03598026  0.1845973 -0.036771847  0.11839537
ENSG00000000938 -0.17548539  0.13802186 -0.1033869  0.120510893 -0.11488039
                00177-0     00181-0     00234-0     00250-0     00266-0     ⋯
ENSG00000000003 -0.14436092 -0.15973690  0.15962816  0.04597725 -0.05037662 ⋯
ENSG00000000419 -0.65566596 -0.07836598 -0.08440686 -0.07336101  0.01694056 ⋯
ENSG00000000457 -0.05600516 -0.11315866 -0.02458583  0.17706909 -0.08072746 ⋯
ENSG00000000460 -0.75807634 -0.07701233  0.05610122 -0.20000546  0.26409694 ⋯
ENSG00000000938 -0.16473877 -0.11642225 -0.04561926 -0.01971724 -0.14657057 ⋯
                39376-0     39390-0     50136-0     62291-0     81801-0     
ENSG00000000003  0.03481634 -0.11353805 -0.07546308 -0.24782141 -0.181882227
ENSG00000000419  0.03302391 -0.17723135 -0.20278517  0.05153863  0.027867915
ENSG00000000457 -0.05381721 -0.06526485  0.06076460  0.07453007  0.118070226
ENSG00000000460  0.08086216  0.04937240 -0.06226473  0.11234436  0.116699931
ENSG00000000938 -0.08194073 -0.11987964 -0.04133224  0.21522278  0.006778182
                84002-0     84689-0      84731-0      84860-0     85365-0    
ENSG00000000003 -0.15831031  0.042505051  0.004158849 -0.04953921 -0.07543666
ENSG00000000419  0.14438615  0.181057380 -0.060171420  0.02275410  0.09297875
ENSG00000000457 -0.02070351 -0.071758383  0.013581220 -0.08240612  0.05418788
ENSG00000000460  0.04048291  0.318198228 -0.003560572  0.09052353  0.07681467
ENSG00000000938  0.10912560 -0.009714053 -0.062825872 -0.17994881  0.04227325

[1] 22236   136

### name to ensemble_id file

In [20]:
#load pickle data to convert gene name to EnsemblID
py_data <- py_load_object("data/input/genes_mapping_id_to_name.pkl")
data_t <- data.frame(t(data.frame(py_data)))
colnames(data_t) <- "gene"
name_to_ID <- rownames_to_column(data_t, var = "EnsemblID")
head(name_to_ID,5)

EnsemblID       gene 
1 ENSG00000000457 SCYL3
2 ENSG00000000938 FGR  
3 ENSG00000000971 CFH  
4 ENSG00000001036 FUCA2
5 ENSG00000001084 GCLC

### metadata file

In [21]:
# Import model metadata file to re-build plier model
metadata <- readRDS("data/input/multiplier_model_metadata.rds")
print(metadata)

$L1
[1] 120.5661

$L2
[1] 241.1322

$L3
[1] 0.01269676



### Multiplier matrices

In [22]:
# load matrices to rebuild plier model
# LV x samples
b_model <- readRDS("data/input/multiplier_model_b.rds")
head(b_model,5)
# gene x LV
z_model <- readRDS("data/input/multiplier_model_z.rds")
head(z_model,5)

SRP000599.SRR013549
1,REACTOME_MRNA_SPLICING                         -0.059296689       
2,SVM Monocytes                                   0.006212678       
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.026105335       
4,REACTOME_NEURONAL_SYSTEM                       -0.022079745       
LV 5                                              0.007663157       
                                                 SRP000599.SRR013550
1,REACTOME_MRNA_SPLICING                         -0.047909034       
2,SVM Monocytes                                   0.003625471       
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.032232060       
4,REACTOME_NEURONAL_SYSTEM                       -0.008970910       
LV 5                                              0.007036176       
                                                 SRP000599.SRR013551
1,REACTOME_MRNA_SPLICING                         -0.049366085       
2,SVM Monocytes                                   0.006604582       
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.020621382       
4,REACTOME_NEURONAL_SYSTEM                       -0.020341711       
LV 5                                              0.006608393       
                                                 SRP000599.SRR013552
1,REACTOME_MRNA_SPLICING                         -0.065078034       
2,SVM Monocytes                                   0.009258006       
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.027598555       
4,REACTOME_NEURONAL_SYSTEM                       -0.016260213       
LV 5                                              0.003446311       
                                                 SRP000599.SRR013553
1,REACTOME_MRNA_SPLICING                         -0.036394186       
2,SVM Monocytes                                   0.005061427       
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.035248076       
4,REACTOME_NEURONAL_SYSTEM                       -0.003022898       
LV 5                                              0.006340665       
                                                 SRP000599.SRR013554
1,REACTOME_MRNA_SPLICING                         -0.046432986       
2,SVM Monocytes                                   0.004132735       
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.038700769       
4,REACTOME_NEURONAL_SYSTEM                        0.002442659       
LV 5                                              0.007106127       
                                                 SRP000599.SRR013555
1,REACTOME_MRNA_SPLICING                         -0.040980500       
2,SVM Monocytes                                   0.008950264       
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.032527087       
4,REACTOME_NEURONAL_SYSTEM                       -0.020457842       
LV 5                                              0.007930485       
                                                 SRP000599.SRR013556
1,REACTOME_MRNA_SPLICING                         -0.040068202       
2,SVM Monocytes                                   0.007226716       
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.030592727       
4,REACTOME_NEURONAL_SYSTEM                       -0.023735309       
LV 5                                              0.009164026       
                                                 SRP000599.SRR013557
1,REACTOME_MRNA_SPLICING                         -0.046137392       
2,SVM Monocytes                                   0.007240987       
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.028937277       
4,REACTOME_NEURONAL_SYSTEM                       -0.021581483       
LV 5                                              0.008023601       
                                                 SRP000599.SRR013558 ⋯
1,REACTOME_MRNA_SPLICING                         -0.048547681        ⋯
2,SVM Monocytes                                   0.005709697        ⋯
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.027405660        ⋯
4,REACTOME_NEURONAL_SYSTEM             

[,1]      [,2] [,3]       [,4] [,5]       [,6]       [,7] [,8]       
GAS6     0.0000000 0    0.03943774 0    0.05047625 0.00000000 0    0.000000000
MMP14    0.0000000 0    0.00000000 0    0.07007159 0.00000000 0    0.004904131
DSP      0.0000000 0    0.00000000 0    0.00000000 0.04169683 0    0.005718149
MARCKSL1 0.3052117 0    0.00000000 0    0.00000000 0.00000000 0    0.000000000
SPARC    0.0000000 0    0.00000000 0    0.00000000 0.00000000 0    0.000000000
         [,9]      [,10]      [,11] [,12]      [,13]      [,14]      
GAS6     0.5909494 0.00000000 ⋯     0.05012510 0.00000000 0.033407371
MMP14    1.7201788 2.42359463 ⋯     0.00000000 0.00000000 0.001007286
DSP      0.0000000 0.00000000 ⋯     0.02085321 0.00000000 0.000000000
MARCKSL1 0.1618435 0.14947148 ⋯     0.02713418 0.05271997 0.000000000
SPARC    0.0000000 0.01401441 ⋯     0.00000000 0.00000000 0.000000000
         [,15]      [,16]      [,17]       [,18]      [,19] [,20]     
GAS6     0.00000000 0.00000000 0.005963392 0.34736209 0     0.00000000
MMP14    0.00000000 0.03574724 0.000000000 0.00000000 0     0.01497801
DSP      0.00000000 0.00000000 0.005774440 0.00000000 0     0.00000000
MARCKSL1 0.03018947 0.06088351 0.000000000 0.00000000 0     0.00000000
SPARC    0.00000000 0.00000000 0.000000000 0.06777859 0     0.12241734
         [,21]    
GAS6     0.0000000
MMP14    0.0000000
DSP      0.4164045
MARCKSL1 0.4484800
SPARC    0.0626649

## label z_matrix with ensemblID instead of gene name

In [23]:
# convert matrix to dataframe with gene as rownames
z_df <- cbind(gene = rownames(z_model), as.data.frame(z_model))
# merge dataframe with 'name_to_ID' (file containing name and ensembleID) by gene
EnsID_df <- merge(z_df, name_to_ID, by = "gene")
#set rownames of new dataframe to ensembleID
rownames(EnsID_df) <- EnsID_df$EnsemblID
#remove gene and ensembleID column
z_matrix <- as.matrix(EnsID_df[, !(colnames(EnsID_df) %in% c("gene", "EnsemblID"))])

str(z_matrix)
head(z_matrix,5)
dim(z_matrix)

 num [1:6455, 1:987] 0 0.2712 0.0051 0.513 1.0082 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:6455] "ENSG00000175899" "ENSG00000094914" "ENSG00000129673" "ENSG00000090861" ...
  ..$ : chr [1:987] "V1" "V2" "V3" "V4" ...


V1          V2        V3         V4         V5 V6       
ENSG00000175899 0.000000000 0.0000000 0.00000000 0.07842750 0  0.0000000
ENSG00000094914 0.271162234 0.0000000 0.00000000 0.00000000 0  0.0000000
ENSG00000129673 0.005098885 0.3084969 0.02881539 0.00000000 0  0.0000000
ENSG00000090861 0.512978085 0.0000000 0.45128547 0.31956792 0  0.0000000
ENSG00000124608 1.008156895 0.0000000 0.19742406 0.02483182 0  0.5708101
                V7        V8         V9        V10       ⋯ V978     V979      
ENSG00000175899 0.0000000 0.04677228 0.0000000 0.0110329 ⋯ 0.010910 0.00000000
ENSG00000094914 0.0000000 0.00000000 1.5133366 0.0000000 ⋯ 0.000000 0.00000000
ENSG00000129673 0.3717253 0.00000000 0.1352375 0.0000000 ⋯ 0.117163 0.01881498
ENSG00000090861 0.0000000 0.00000000 0.0000000 0.0000000 ⋯ 0.000000 0.00000000
ENSG00000124608 0.0000000 0.22867460 0.0000000 0.1162435 ⋯ 0.000000 0.00000000
                V980        V981        V982 V983       V984       V985      
ENSG00000175899 0.052869436 0.012748534 0    0.01398231 0.15224090 0.00000000
ENSG00000094914 0.000000000 0.001385031 0    0.00000000 0.71189565 0.03053447
ENSG00000129673 0.000000000 0.193141835 0    0.04040077 0.18043565 0.00000000
ENSG00000090861 0.006207784 0.021274385 0    0.00000000 0.06214269 0.00000000
ENSG00000124608 0.153344992 0.000000000 0    0.00000000 1.00457811 0.01447016
                V986      V987      
ENSG00000175899 0.0000000 0.10888358
ENSG00000094914 0.4471055 0.42468522
ENSG00000129673 0.0000000 0.04206449
ENSG00000090861 0.0000000 0.36489213
ENSG00000124608 0.0000000 0.00000000

[1] 6455  987

In [24]:
# rebuild plier model with z, b, L1, L2, L3
plier.model <- list(Z = z_matrix, B =b_model, L1 = metadata$L1, L2 = metadata$L2, L3 = metadata$L3)

In [25]:
# Check all variables are loaded
ls()

[1] "b_model"                      "CalculateHoldoutAUC"         
 [3] "CalculateUSparsity"           "data_t"                      
 [5] "df_projection"                "EnsID_df"                    
 [7] "EvalWrapper"                  "EvalWrapperWithHoldout"      
 [9] "GetNewDataB"                  "GetOrderedRowNorm"           
[11] "GetOrderedRowNormEM"          "GetPathwayCoverage"          
[13] "GetReconstructedExprs"        "GetReconstructionCorrelation"
[15] "GetReconstructionMASE"        "metadata"                    
[17] "name_to_ID"                   "ordered_DAISY_gen_expr"      
[19] "ordered_mat"                  "ordered_model"               
[21] "path_to_rna"                  "plier.model"                 
[23] "PLIERNewData"                 "projection"                  
[25] "py_data"                      "RNAseq_1_res"                
[27] "z_df"                         "z_matrix"                    
[29] "z_model"

In [26]:
#order gene expression matrix to match plier model
ordered_mat <- GetOrderedRowNormEM(RNAseq_1_res, plier.model)

#extract gene_expressiom matrix
ordered_DAISY_gen_expr <- ordered_mat$exprs.norm.filtered

# extract plier model matrix
ordered_model <- ordered_mat$plier.model

In [27]:
#project gen expression matrix into plier model
projection <- GetNewDataB(ordered_DAISY_gen_expr,ordered_model)

In [28]:
#save projection
df_projection  <- as.data.frame(projection)
saveRDS(df_projection,"output/projection_1.rds")

In [29]:
dim(df_projection)

[1] 987 136

In [30]:
head(df_projection, 10)

00041-0      00110-0    
1,REACTOME_MRNA_SPLICING                          0.045748028 -0.08426191
2,SVM Monocytes                                   0.104307804  0.02627734
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.034338781 -0.02141530
4,REACTOME_NEURONAL_SYSTEM                        0.056377294  0.04507367
LV 5                                             -0.006014439  0.00770212
LV 6                                              0.013141592  0.04091529
7,IRIS_Neutrophil-Resting                        -0.158699344  0.06129902
8,KEGG_PPAR_SIGNALING_PATHWAY                    -0.014859012  0.03218188
LV 9                                             -0.032401545 -0.08186520
10,SVM Macrophages M0                            -0.012606430  0.01204398
                                                 00139-0      00159-0     
1,REACTOME_MRNA_SPLICING                         -0.077697184 -0.078539199
2,SVM Monocytes                                  -0.049913741  0.266457766
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.102291965 -0.004261070
4,REACTOME_NEURONAL_SYSTEM                        0.097356702  0.041259268
LV 5                                             -0.023535277  0.010785959
LV 6                                              0.033234436 -0.026479309
7,IRIS_Neutrophil-Resting                        -0.067108808  0.043252439
8,KEGG_PPAR_SIGNALING_PATHWAY                     0.008868332  0.005756415
LV 9                                             -0.086891463 -0.028086861
10,SVM Macrophages M0                            -0.008437336  0.030484368
                                                 00174-0      00177-0     
1,REACTOME_MRNA_SPLICING                          0.022015152  0.114903283
2,SVM Monocytes                                  -0.008530261 -0.048283019
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.006897807  0.043636271
4,REACTOME_NEURONAL_SYSTEM                        0.027453577 -0.041370318
LV 5                                              0.017448294 -0.060001645
LV 6                                             -0.004379551  0.061410724
7,IRIS_Neutrophil-Resting                        -0.080901625 -0.089031326
8,KEGG_PPAR_SIGNALING_PATHWAY                    -0.007961799 -0.100436616
LV 9                                             -0.013467576  0.007585578
10,SVM Macrophages M0                             0.012146043 -0.061066100
                                                 00181-0      00234-0      
1,REACTOME_MRNA_SPLICING                          0.033916849 -0.0007790358
2,SVM Monocytes                                  -0.052436882 -0.0500619312
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES -0.041807814  0.0247366996
4,REACTOME_NEURONAL_SYSTEM                        0.049110456  0.0497693154
LV 5                                             -0.019730027  0.0274240952
LV 6                                              0.008241706  0.0501137283
7,IRIS_Neutrophil-Resting                        -0.066598034 -0.0180259054
8,KEGG_PPAR_SIGNALING_PATHWAY                     0.026326809  0.0203580538
LV 9                                             -0.007937155  0.0006474471
10,SVM Macrophages M0                            -0.041958257 -0.0264450820
                                                 00250-0       00266-0      ⋯
1,REACTOME_MRNA_SPLICING                         -0.0550538495  0.307223472 ⋯
2,SVM Monocytes                                   0.1014881826 -0.106715583 ⋯
3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES  0.0033781163 -0.040374505 ⋯
4,REACTOME_NEURONAL_SYSTEM                        0.0498141323 -0.022980039 ⋯
LV 5                                              0.0753578701  0.002400883 ⋯
LV 6                                              0.0313095320 -0.003194558 ⋯
7,IRIS_Neutrophil-Resting                         0.0072873850 -0.059118798 ⋯
8,KEGG_PPAR_SIGNALING_PATHWAY                    -0.0004896021  0.006190327 ⋯
LV 9                                           